Map of Irrigated Agricultural Areas	foo.028	http://www.fao.org/nr/water/aquastat/irrigationmap/index10.stm

foo.028 http://www.fao.org/nr/water/aquastat/irrigationmap/index10.stm
Downloaded to RW_Data/Rasters/aquastat_map_irrigated_areas
File type: asc

Import libraries

In [2]:
# Libraries for downloading data from remote server (may be ftp)
import requests
from urllib.request import urlopen
from contextlib import closing
import shutil

# Library for uploading/downloading data to/from S3
import boto3

# Libraries for handling data
import rasterio as rio
import numpy as np
# from netCDF4 import Dataset
# import pandas as pd
# import scipy

# Libraries for various helper functions
# from datetime import datetime
import os
import threading
import sys
from glob import glob

s3 tools

In [28]:
s3_upload = boto3.client("s3")
s3_download = boto3.resource("s3")

s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/foo_028_aquastat_irrigated_areas/"

s3_file1 = "foo_028_aquastat_irrigated_areas_area_equipped_irrigation.asc"
s3_file2 = "foo_028_aquastat_irrigated_areas_aai_pct_aei.asc"
s3_file3 = "foo_028_aquastat_irrigated_areas_aeigw_pct_aei.asc"
s3_file4 = "foo_028_aquastat_irrigated_areas_aeinc_pct_aei.asc"
s3_file5 = "foo_028_aquastat_irrigated_areas_aeisw_pct_aei.asc"

s3_key_orig1 = s3_folder + s3_file1
s3_key_edit1 = s3_key_orig1[0:-4] + "_edit.tif"

s3_key_orig2 = s3_folder + s3_file2
s3_key_edit2 = s3_key_orig2[0:-4] + "_edit.tif"

s3_key_orig3 = s3_folder + s3_file3
s3_key_edit3 = s3_key_orig3[0:-4] + "_edit.tif"

s3_key_orig4 = s3_folder + s3_file4
s3_key_edit4 = s3_key_orig4[0:-4] + "_edit.tif"

s3_key_orig5 = s3_folder + s3_file5
s3_key_edit5 = s3_key_orig5[0:-4] + "_edit.tif"

class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write("\r%s  %s / %s  (%.2f%%)"%(
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

Define local file locations

In [18]:
local_folder = "/Users/nathansuberi/Desktop/RW_Data/Rasters/aquastat_map_irrigated_areas/"

file_name1 = "gmia_v5_aei_ha.asc"
file_name2 = "gmia_v5_aai_pct_aei.asc"
file_name3 = "gmia_v5_aeigw_pct_aei.asc"
file_name4 = "gmia_v5_aeinc_pct_aei.asc"
file_name5 = "gmia_v5_aeisw_pct_aei.asc"

local_orig1 = local_folder + file_name1
local_orig2 = local_folder + file_name2
local_orig3 = local_folder + file_name3
local_orig4 = local_folder + file_name4
local_orig5 = local_folder + file_name5

orig_extension_length = 4 #4 for each char in .tif

local_edit1 = local_orig1[:-orig_extension_length] + "_edit.tif" 
local_edit2 = local_orig2[:-orig_extension_length] + "_edit.tif" 
local_edit3 = local_orig3[:-orig_extension_length] + "_edit.tif" 
local_edit4 = local_orig4[:-orig_extension_length] + "_edit.tif" 
local_edit5 = local_orig5[:-orig_extension_length] + "_edit.tif" 

Use rasterio to reproject and compress

In [19]:
with rio.open(local_orig1, 'r') as src:
    profile = src.profile
    print(profile)
    data = src.read()
    

{'driver': 'AAIGrid', 'dtype': 'float32', 'nodata': -9.0, 'width': 4320, 'height': 2160, 'count': 1, 'crs': CRS({}), 'transform': (-180.0, 0.08333333333333333, 0.0, 90.0, 0.0, -0.08333333333333333), 'affine': Affine(0.08333333333333333, 0.0, -180.0,
       0.0, -0.08333333333333333, 90.0), 'tiled': False}


In [21]:
data[0]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [22]:
# Note - this is the core of Vizz's netcdf2tif function

def convert_asc_to_tif(orig_name, edit_name):
    with rio.open(orig_name, 'r') as src:
        # This assumes data is readable by rasterio
        # May need to open instead with netcdf4.Dataset, for example

        data = src.read()[0]

        rows = data.shape[0]
        columns = data.shape[1]

        print(rows)
        print(columns)

        # Latitude bounds
        south_lat = -90
        north_lat = 90

        # Longitude bounds
        west_lon = -180
        east_lon = 180

        transform = rio.transform.from_bounds(west_lon, south_lat, east_lon, north_lat, columns, rows)

        # Profile

        no_data_val = -9
        target_projection = 'EPSG:4326'
        target_data_type = np.float32

        profile = {
            'driver':'GTiff', 
            'height':rows, 
            'width':columns, 
            'count':1, 
            'dtype':target_data_type, 
            'crs':target_projection, 
            'transform':transform, 
            'compress':'lzw', 
            'nodata': no_data_val
        }

        with rio.open(edit_name, "w", **profile) as dst:
            dst.write(data.astype(profile["dtype"]), 1)

In [24]:
convert_asc_to_tif(local_orig1, local_edit1)
convert_asc_to_tif(local_orig2, local_edit2)
convert_asc_to_tif(local_orig3, local_edit3)
convert_asc_to_tif(local_orig4, local_edit4)
convert_asc_to_tif(local_orig5, local_edit5)

2160
4320
2160
4320
2160
4320
2160
4320
2160
4320


In [27]:
local_orig1

'/Users/nathansuberi/Desktop/RW_Data/Rasters/aquastat_map_irrigated_areas/gmia_v5_aei_ha.asc'

Upload orig and edit files to s3

In [29]:
# Original
s3_upload.upload_file(local_orig1, s3_bucket, s3_key_orig1,
                         Callback=ProgressPercentage(local_orig1))
s3_upload.upload_file(local_orig2, s3_bucket, s3_key_orig2,
                         Callback=ProgressPercentage(local_orig2))
s3_upload.upload_file(local_orig3, s3_bucket, s3_key_orig3,
                         Callback=ProgressPercentage(local_orig3))
s3_upload.upload_file(local_orig4, s3_bucket, s3_key_orig4,
                         Callback=ProgressPercentage(local_orig4))
s3_upload.upload_file(local_orig5, s3_bucket, s3_key_orig5,
                         Callback=ProgressPercentage(local_orig5))

# Edit
s3_upload.upload_file(local_edit1, s3_bucket, s3_key_edit1,
                         Callback=ProgressPercentage(local_edit1))
s3_upload.upload_file(local_edit2, s3_bucket, s3_key_edit2,
                         Callback=ProgressPercentage(local_edit2))
s3_upload.upload_file(local_edit3, s3_bucket, s3_key_edit3,
                         Callback=ProgressPercentage(local_edit3))
s3_upload.upload_file(local_edit4, s3_bucket, s3_key_edit4,
                         Callback=ProgressPercentage(local_edit4))
s3_upload.upload_file(local_edit5, s3_bucket, s3_key_edit5,
                         Callback=ProgressPercentage(local_edit5))

/Users/nathansuberi/Desktop/RW_Data/Rasters/aquastat_map_irrigated_areas/gmia_v5_aeisw_pct_aei_edit.tif  2527616 / 2527616.0  (100.00%)